# Prepare

In [ ]:
import os 

In [ ]:
input_path = 'spinn_hmi'
input_original = os.path.join(input_path, 'original')
os.makedirs(input_original, exist_ok=True)

In [ ]:
bin = 2
b_norm = 2500
spatial_norm = 160
nz = 160

## Prepare

In [ ]:
import glob
from sunpy.map import Map
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt 

In [ ]:
data_path = '/home/tensor/workspace/GPUserver/workspace/NF2/example/AR_7115'

hmi_p = sorted(glob.glob(os.path.join(data_path, '*Bp.fits')))[0]  # x
hmi_t = sorted(glob.glob(os.path.join(data_path, '*Bt.fits')))[0]  # y
hmi_r = sorted(glob.glob(os.path.join(data_path, '*Br.fits')))[0]  # z

p_map, t_map, r_map = Map(hmi_p), Map(hmi_t), Map(hmi_r)

maps = [p_map, t_map, r_map]

maps = [m.superpixel(u.Quantity((bin, bin), u.pixel), func=np.mean) for m in maps]

b_bottom = np.stack([maps[0].data, -maps[1].data, maps[2].data]).transpose()

In [ ]:
plt.imshow(b_bottom[:, :, 2].transpose(), origin='lower')

In [ ]:
import pickle

In [ ]:
b_bottom_original_path = os.path.join(input_original, "b_bottom_original.pickle")

with open(b_bottom_original_path,"wb") as f:
    pickle.dump(b_bottom, f)

In [ ]:
nx, ny, _ = b_bottom.shape

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm
from zpinn.pinn_nf2_cleanup import PotentialModel, create_coordinates

bottom_values = b_bottom.reshape(-1, 3)
bottom_bounds = (0, nx-1, 0, ny-1, 0, 0)
bottom_coords = create_coordinates(bottom_bounds).reshape(-1, 3)

top_bounds = (0, nx-1, 0, ny-1, nz-1, nz-1)
lateral_bounds_1 = (0, 0, 0, ny-1, 0, nz-1)
lateral_bounds_2 = (nx-1, nx-1, 0, ny-1, 0, nz-1)
lateral_bounds_3 = (0, nx-1, 0, 0, 0, nz-1)
lateral_bounds_4 = (0, nx-1, ny-1, ny-1, 0, nz-1)

top_coords = create_coordinates(top_bounds)
lateral_coords_1 = create_coordinates(lateral_bounds_1)
lateral_coords_2 = create_coordinates(lateral_bounds_2)
lateral_coords_3 = create_coordinates(lateral_bounds_3)
lateral_coords_4 = create_coordinates(lateral_bounds_4)

top_lateral_coordinates = [top_coords,
                           lateral_coords_1,
                           lateral_coords_2,
                           lateral_coords_3,
                           lateral_coords_4]

In [ ]:
from zpinn.create_pot import get_potential_boundary

In [ ]:
coords, fields = get_potential_boundary(b_bottom[:, :, 2], nz)

In [ ]:
bp_top = fields[0]
bp_lateral_3 = fields[1]
bp_lateral_4 = fields[2]
bp_lateral_1 = fields[3]
bp_lateral_2 = fields[4]

In [ ]:
b_bottom = b_bottom / b_norm
bp_top = bp_top / b_norm
bp_lateral_1 = bp_lateral_1 / b_norm
bp_lateral_2 = bp_lateral_2 / b_norm
bp_lateral_3 = bp_lateral_3 / b_norm 
bp_lateral_4 = bp_lateral_4 / b_norm

In [ ]:
for i in range(len(coords)):
    coords[i] = np.reshape(coords[i], (-1, 3))
    fields[i] = np.reshape(fields[i], (-1, 3))

top_lateral_values = np.concatenate(fields) 
top_lateral_coords = np.concatenate(coords)

boundary_values = np.concatenate([top_lateral_values, bottom_values])
boundary_coords = np.concatenate([top_lateral_coords, bottom_coords])

normalized_boundary_values = boundary_values / b_norm
normalized_boundary_coords = boundary_coords / spatial_norm

boundary_data = np.stack([normalized_boundary_coords, normalized_boundary_values], 1)

In [ ]:
b_bottom_path = os.path.join(input_path, "b_bottom.pickle")
bp_top_path = os.path.join(input_path, "bp_top.pickle")
bp_lateral_1_path = os.path.join(input_path, "bp_lateral_1.pickle")
bp_lateral_2_path = os.path.join(input_path, "bp_lateral_2.pickle")
bp_lateral_3_path = os.path.join(input_path, "bp_lateral_3.pickle")
bp_lateral_4_path = os.path.join(input_path, "bp_lateral_4.pickle")
boundary_data_path = os.path.join(input_path, "boundary_data.pickle")

In [ ]:
with open(b_bottom_path,"wb") as f:
    pickle.dump(b_bottom, f)

with open(bp_top_path,"wb") as f:
    pickle.dump(bp_top, f)

with open(bp_lateral_1_path,"wb") as f:
    pickle.dump(bp_lateral_1, f)

with open(bp_lateral_2_path,"wb") as f:
    pickle.dump(bp_lateral_2, f)
    
with open(bp_lateral_3_path,"wb") as f:
    pickle.dump(bp_lateral_3, f)

with open(bp_lateral_4_path,"wb") as f:
    pickle.dump(bp_lateral_4, f)

with open(boundary_data_path,"wb") as f:
    pickle.dump(boundary_data, f)